In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
pip install "vllm[cuda118]"

In [4]:
pip install pymupdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 28.8 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
!pip install accelerate --upgrade

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 8.6 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.7 MB/s eta 0:00:0000:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pdfplumber
import pandas as pd
import fitz
def extract_text_pymupdf(pdf_path):
    """Extracts text from a structured (non-scanned) PDF."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

In [2]:
def extract_tables_pdf(pdf_path):
    """Extracts tables from a PDF and converts them into DataFrames."""
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                df = pd.DataFrame(table[1:], columns=table[0])  # First row as headers
                tables.append(df)
    return tables

In [3]:
def extract_images_pdf(pdf_path, save_folder="images"):
    """Extracts rasterized page images from a PDF and saves them as PNG files."""
    import os
    import pdfplumber

    images = []
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            img = page.to_image(resolution=150).original  # PIL Image object
            img_filename = os.path.join(save_folder, f"pdf_page_{i+1}.png")
            img.save(img_filename, "PNG")
            images.append(img_filename)

    return images


In [4]:
import requests
from bs4 import BeautifulSoup

def extract_text_webpage(url):
    """Extracts text from an article webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = "\n".join([p.get_text(strip=True) for p in soup.find_all("p")])
    return text


In [5]:
def extract_tables_webpage(url):
    """Extracts tables from a webpage and converts them into DataFrames."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    tables = []
    for table in soup.find_all("table"):
        rows = []
        for row in table.find_all("tr"):
            cells = [cell.get_text(strip=True) for cell in row.find_all(["td", "th"])]
            rows.append(cells)
        if rows:
            df = pd.DataFrame(rows[1:], columns=rows[0])  # First row as headers
            tables.append(df)
    return tables

In [6]:
def extract_images_webpage(url, save_folder="images"):
    """Extracts images from a webpage and saves them locally."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    images = []
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    for i, img in enumerate(soup.find_all("img")):
        img_url = img.get("src")
        if img_url and img_url.startswith(("http", "//")):
            img_url = img_url if img_url.startswith("http") else "https:" + img_url
            try:
                img_data = requests.get(img_url).content
                img_filename = os.path.join(save_folder, f"image_{i+1}.jpg")
                with open(img_filename, "wb") as f:
                    f.write(img_data)
                images.append(img_filename)
            except Exception as e:
                print(f"Failed to download {img_url}: {e}")
    return images

In [7]:
def extract_text(source):
    """Extracts text from PDF or web URL."""
    if source.endswith(".pdf"):
        return extract_text_pymupdf(source)  # Extract from PDF
    elif source.startswith("http"):
        return extract_text_webpage(source)  # Extract from web page
    else:
        raise ValueError("Unsupported file type. Provide a PDF or URL.")


In [8]:
def extract_tables(source):
    """Extracts tables from a PDF or web URL and converts them into DataFrames."""
    if source.endswith(".pdf"):
        return extract_tables_pdf(source)  # Extract from PDF
    elif source.startswith("http"):
        return extract_tables_webpage(source)  # Extract from web page
    else:
        raise ValueError("Unsupported file type. Provide a PDF or URL.")

In [9]:
def extract_images(source, save_folder="images"):
    """Extracts images from PDFs or web pages and saves them locally."""
    if source.endswith(".pdf"):
        return extract_images_pdf(source, save_folder)  # Extract from PDF
    elif source.startswith("http"):
        return extract_images_webpage(source, save_folder)  # Extract from web page
    else:
        raise ValueError("Unsupported file type. Provide a PDF or URL.")

In [10]:
from transformers import pipeline

def extract_all(sources):
    """Extracts text, tables, and images from multiple PDFs and web pages."""
    combined_text = []
    combined_tables = []
    combined_images = []
    source_references = []  # Store source details

    for source in sources:
        text = extract_text(source)
        tables = extract_tables(source)
        images = extract_images(source)

        combined_text.append(text)
        combined_tables.extend(tables)
        combined_images.extend(images)
        source_references.append(source)

    return combined_text, combined_tables, combined_images, source_references

/home/jupyter-wem220/.local/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-04-03 01:48:44.805702: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 01:48:44.821526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 01:48:44.841753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

def generate_lesson_from_extracted_data(sources):
    """Extracts data from multiple sources and generates an AI-powered lesson with embedded tables and image placeholders."""

    # Step 1: Extract content
    all_texts, all_tables, all_images, all_sources = extract_all(sources)

    # Step 2: Convert tables to Markdown
    def table_to_markdown(tables):
        markdown_tables = []
        for df in tables:
            try:
                markdown_tables.append(df.to_markdown(index=False))
            except Exception as e:
                markdown_tables.append(f"*Table could not be rendered: {e}*")
        return "\n\n".join(markdown_tables)

    tables_markdown = table_to_markdown(all_tables)

    # Step 3: Prepare combined text
    combined_text = "\n\n".join(all_texts)[:4000]  # Truncate to fit token limits

    # Step 4: Image markdown placeholders
    image_placeholders = "\n".join([
        f"![image]({img})" for img in all_images
    ]) if all_images else "No images extracted."

    # Step 5: Construct the prompt
    prompt = f"""
You are an educational content generator. Your task is to create a structured lesson using the extracted text, tables, and images provided below.

### Extracted Text:
{combined_text}

### Extracted Tables (Markdown Format):
{tables_markdown}

### Source Webpages & Documents:
{', '.join(all_sources)}

### Extracted Images:
Use the following image files at appropriate points in your lesson. Include each one using Markdown syntax like: `![image](imageX.png)`.

{image_placeholders}

---

### 📝 Instructions:
- Integrate the **Markdown tables** into the lesson where they add value.
- **Include each image** by referencing it at the most relevant point in the content (using the Markdown format).
- Write a clear, engaging lesson with sections, subheadings, and explanations.
- Briefly describe the images where included, to help learners understand their relevance.

Begin generating the lesson below:
"""

    # Step 6: Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-chat-hf",
        device_map="auto",
        torch_dtype=torch.float16
    )

    # Step 7: Tokenize and check input
    encoded = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
    input_ids = encoded["input_ids"]

    if torch.isnan(input_ids).any() or torch.isinf(input_ids).any():
        raise ValueError("Input prompt contains NaN or Inf values!")

    # Step 8: Generate only new text
    output_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=1024,  # More room for generation
        do_sample=False,      # Greedy decoding for now
        temperature=0.7,
        return_dict_in_generate=True,
        output_scores=True
    )

    # Step 9: Strip the prompt to get only the new tokens
    generated_ids = output_ids.sequences[0][input_ids.shape[-1]:]
    output = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # Debugging: Show intermediate info
    print("✅ Prompt Length:", input_ids.shape[-1])
    print("✅ Tokens Generated:", len(generated_ids))
    print("✅ Output Preview:\n", output[:500])  # Preview first 500 characters

    # Optional: Save to file for debugging
    with open("lesson_output.txt", "w", encoding="utf-8") as f:
        f.write(output)

    return output


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pdf_path = "Classification in Machine Learning - Python Geeks.pdf"

# Extract content
sources = [pdf_path]
text, tables, images, source_refs = extract_all(sources)
lesson = generate_lesson_from_extracted_data(sources)
print(lesson if lesson.strip() else "❌ No content was generated.")


In [23]:
!nvidia-smi

Thu Apr  3 01:40:13 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               Off | 00000000:18:00.0 Off |                  Off |
| 30%   29C    P8              20W / 230W |  24176MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
print(lesson)